In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.layers import Dropout
from keras import regularizers
import numpy as np
from keras.datasets import imdb
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%matplotlib inline
import matplotlib.pyplot as plt
from data_reader import load_data, DataReader

Using TensorFlow backend.


In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
X_train = np.load('x_train.npz')['arr_0']
#y_train = np.load('train_genres.npz')['arr_0']
y_train = np.load('y_train.npz')['arr_0']

X_val = np.load('x_valid.npz')['arr_0']
#y_val= np.load('valid_genres.npz')['arr_0']
y_val = np.load('y_valid.npz')['arr_0']
X_test = np.load('x_test.npz')['arr_0']
y_test= np.load('y_test.npz')['arr_0']
#y_test =np.load('test_genres.npz')['arr_0']

In [3]:
def create_hist(data, vocab_size):
    hist = np.zeros((data.shape[0],vocab_size))
    count = 0
    for item in data:

        for i in item:
            hist[count,i] += 1
        count += 1
    return hist


In [4]:
hist_train = create_hist(X_train, 60000)

hist_val =create_hist(X_val, 60000)
hist_test =create_hist(X_test, 60000)



In [5]:
vocab = np.load('vocab.npz')['arr_0']
vocab_counts = np.load('vocab_counts.npz')['arr_0']
for i in range(10):
    print(vocab[i])
print(vocab_counts[0:10])
word_vocab, char_vocab, _, _, _ = \
        load_data('data', 35, eos='+')

bir
i̇ngiliz
subayı
mısır
da
bulunan
kaleye
komutan
olarak
atanmış
[9736  291    6   22 2119   40    3  135  238    3]
reading train
reading valid
reading test

actual longest token length is: 34
size of word vocabulary: 56938
size of char vocabulary: 63
number of tokens in train: 259760
number of tokens in valid: 67092
number of tokens in test: 82430


In [6]:
embeddings = np.load('wembeddings300.npz')['arr_0']
vocab_size = embeddings.shape[0]
sort_index = np.flip(np.argsort(vocab_counts),0)

new_embeddings = np.zeros((vocab.shape[0],300))
for i in range(vocab.shape[0]):
    if i%10000 == 0:
        print(i)
    if word_vocab.get(vocab[sort_index[i]]) is not None:
        new_embeddings[i,:] = embeddings[word_vocab.get(vocab[sort_index[i]]),:]
    else:
        print(i)
        new_embeddings[i,:] = np.random.randn(1,300)
    


0
5306
10000
15726
15894
19970
20000
23332
24793
24809
25324
25630
28930
29735
30000
32317
40000
40428
40696
40697
40698
40699
40700
40701
40702
40707
40708
40716
40718
40720
40721
40723
40725
40727
40728
40729
40730
40733
41138
44948
46961
50000


In [5]:
print(word_vocab.get(vocab[i]))

51053


In [6]:
y_train_binary = np.array(y_train > 6).astype(int)
y_val_binary = np.array(y_val > 6).astype(int)
y_test_binary = np.array(y_test > 6).astype(int)

In [ ]:
batch_size = 64
def batch_gen():
    for i in range(X_train.shape[0]//batch_size):
        X = X_train[i*batch_size:(i+1)*batch_size]
        ls = []
        for i in X:
            ls.append(len(i))
        
        X = sequence.pad_sequences(X, maxlen=max(ls))
        #Y = y_train[i*batch_size:(i+1)*batch_size]
        yield X, y_train


In [ ]:
y_train.shape
embeddings = np.load('wembeddings300.npz')['arr_0']
vocab_size = embeddings.shape[0]

In [14]:
#y_train = np.load('y_train.npz')['arr_0']
y_train_one = to_categorical(y_train)
y_valid_one = to_categorical(y_val)
y_test_one = to_categorical(y_test)

In [23]:


embedding_vecor_length = 300
max_fikra_length = 150
X_t = sequence.pad_sequences(X_train, maxlen=max_fikra_length)
X_v = sequence.pad_sequences(X_val, maxlen=max_fikra_length)

model = Sequential()
model.add(Embedding(new_embeddings.shape[0], embedding_vecor_length, input_length=max_fikra_length, embeddings_initializer=keras.initializers.Constant(value=new_embeddings)))
model.add(Dropout(0.8))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_t, y_train_one, validation_data=(X_v, y_valid_one), epochs=10, batch_size=32,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 150, 300)          15091200  
_________________________________________________________________
dropout_4 (Dropout)          (None, 150, 300)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               160400    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
Total params: 15,252,610
Trainable params: 15,252,610
Non-trainable params: 0
_________________________________________________________________
None
Train on 2653 samples, validate on 755 samples
Epoch 1/10
2653/2653 [==============================] - 19s 7ms/step - loss: 2.0526 - acc: 0.2544 - val_loss: 1.9281 - val_acc: 0.2887

KeyboardInterrupt: 

In [17]:
embedding_vecor_length = 300
max_fikra_length = 600
X_t = sequence.pad_sequences(X_train, maxlen=max_fikra_length)
X_v = sequence.pad_sequences(X_val, maxlen=max_fikra_length)

model = Sequential()
model.add(Embedding(new_embeddings.shape[0], embedding_vecor_length, input_length=max_fikra_length, embeddings_initializer=keras.initializers.Constant(value=new_embeddings)))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(30))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_t, y_train_binary, validation_data=(X_v, y_val_binary), epochs=10, batch_size=64,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 600, 300)          15091200  
_________________________________________________________________
lstm_9 (LSTM)                (None, 600, 100)          160400    
_________________________________________________________________
lstm_10 (LSTM)               (None, 30)                15720     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 31        
Total params: 15,267,351
Trainable params: 15,267,351
Non-trainable params: 0
_________________________________________________________________
None
Train on 2653 samples, validate on 755 samples
Epoch 1/10
2653/2653 [==============================] - 70s 26ms/step - loss: 0.6795 - acc: 0.5646 - val_loss: 0.6768 - val_acc: 0.5576

KeyboardInterrupt: 

In [14]:
scores = model.evaluate(X_v, y_val_binary, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
model.fit(X_t, y_train_binary, validation_data=(X_v, y_val_binary), epochs=10, batch_size=64,verbose=1)

Accuracy: 58.28%
Train on 2653 samples, validate on 755 samples
Epoch 1/10
2653/2653 [==============================] - 17s 6ms/step - loss: 0.0255 - acc: 0.9894 - val_loss: 1.5541 - val_acc: 0.5788

KeyboardInterrupt: 

In [ ]:

model = Sequential()
model.add(Dense(100,input_dim=60000, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(hist_train, y_train_binary, validation_data=(hist_val, y_val_binary), epochs=10, batch_size=32 ,verbose=0)

In [ ]:
# create the model
embedding_vecor_length = 300
model = Sequential()
#model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length, embeddings_initializer=keras.initializers.Constant(value=embeddings)))
model.add(Dense(100,input_dim=60000, activation='relu'))
model.add(Dense(19, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(hist_train, y_train_one, validation_data=(hist_val, y_valid_one), epochs=10, batch_size=128,verbose=0)

In [ ]:
# Final evaluation of the model
print(hist_test.shape)
scores = model.evaluate(hist_val, y_valid_one, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:

output = model.predict(hist_val)
print(output.shape)
predictions = np.argmax(output,1)
print(float(np.sum(y_val == predictions)))/y_val.shape[0]

In [3]:
y_train = np.load('train_genres.npz')['arr_0']
y_val= np.load('valid_genres.npz')['arr_0']

(values,counts) = np.unique(y_val,return_counts=True)
ind=np.argmax(counts)
print(ind)

16


In [4]:
float(counts[16])/np.sum(counts)

0.22781456953642384

In [ ]:
values
counts
k = 7
print(float(np.sum(counts[0:k]))/np.sum(counts[0:10]))


In [ ]:
val_outs = model.predict(X_val)

In [ ]:
preds = np.argmax(val_outs,1)
print(float(np.sum(preds == y_val)))/len(y_val)